In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
Data = pd.read_csv('../input/covid-rna-seq/COVID_RNA_Seq.csv', index_col=0).T

In [ ]:
Data.info()

In [ ]:
Data['label'] = 1
Data['label'].iloc[100:126]=0
X = Data.drop('label', axis=1)
y = Data['label']

In [ ]:
from sklearn.model_selection import train_test_split
X_, X_val, y_, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)

XGBoost

In [ ]:
import xgboost as xgb
from sklearn.metrics import classification_report
clf = xgb.XGBClassifier()
clf.fit(X_,y_)
y_pred = clf.predict(X_val)

In [ ]:
print(classification_report(y_val,y_pred))
from sklearn.metrics import confusion_matrix
confusion_matrix(y_val,y_pred)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

stratifiedkfold = StratifiedKFold(n_splits=5)
scores = cross_val_score(clf, X ,y,cv=stratifiedkfold)
print('Cross-validation scores: \n{}'.format(scores))

In [ ]:
import numpy as np
print('Average score: {}'.format(np.mean(scores)))

In [ ]:
import matplotlib.pyplot as plt

_, ax = plt.subplots(figsize=(12, 4))
xgb.plot_importance(clf,
                        ax=ax,
                        importance_type='gain',
                        show_values=False,
                        max_num_features=5)
plt.show()

According to the ensembl website(http://www.ensembl.org/index.html), 

* > ENST00000431458 →　 *FTLP2*
* > ENST00000390304 →　*IGLV3-27*
* > ENST00000652660 →　*RRM2*
* > ENST00000445125 →　*AC010970.1*
* > ENST00000344318 →　*ZMAT5*

LR

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression # Logistic Regression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict

LR = LogisticRegression()
LR.fit(X_,y_)
prediction_lr=LR.predict(X_val)
print('--------------The Accuracy of the model----------------------------')
print('The accuracy of the Logistic Regression is',round(accuracy_score(prediction_lr,y_val)*100,2))
kfold = KFold(n_splits=5, random_state=0) # k=5, split the data into 10 equal parts
result_lr=cross_val_score(LR,X,y,cv=5,scoring='accuracy')
print('The cross validated score for Logistic REgression is:',round(result_lr.mean()*100,2))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(LR,X,y,cv=5)
sns.heatmap(confusion_matrix(y,y_pred),annot=True,fmt='3.0f',cmap="summer")
plt.title('Confusion_matrix(LR)', y=1.05, size=15)

RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=0, verbose=0,
            warm_start=False)
RF.fit(X_,y_)
prediction_rm=RF.predict(X_val)
print('--------------The Accuracy of the model----------------------------')
print('The accuracy of the Random Forest Classifier is',round(accuracy_score(prediction_rm,y_val)*100,2))

In [ ]:
kfold = KFold(n_splits=5, random_state=0) # k=5, split the data into 5 equal parts
result_rm=cross_val_score(RF,X,y,cv=5,scoring='accuracy')
print('The cross validated score for Random Forest Classifier is:',round(result_rm.mean()*100,2))
y_pred = cross_val_predict(RF,X,y,cv=5)
sns.heatmap(confusion_matrix(y,y_pred),annot=True,fmt='3.0f',cmap="summer")
plt.title('Confusion_matrix(RF)', y=1.05, size=15)

In [ ]:
import matplotlib.pyplot as plt
features = X_.columns
importances = RF.feature_importances_
indices = np.argsort(importances)[::-1][0:5][::-1]

plt.figure(figsize= (10,5))
plt.barh(range(len(indices)), importances[indices])
plt.yticks(range(len(indices)),features[indices])
plt.show()

Emsembl website(http://grch37.ensembl.org/index.html) will tell you the related gene.

SVM

In [ ]:
from sklearn.svm import SVC, LinearSVC

SVM = SVC()
SVM.fit(X_,y_)
prediction_svm=SVM.predict(X_val)
print('--------------The Accuracy of the model----------------------------')
print('The accuracy of the Support Vector Machines Classifier is',round(accuracy_score(prediction_svm,y_val)*100,2))
kfold = KFold(n_splits=5, random_state=22) # k=10, split the data into 10 equal parts
result_svm=cross_val_score(SVM,X,y,cv=5,scoring='accuracy')
print('The cross validated score for Support Vector Machines Classifier is:',round(result_svm.mean()*100,2))
y_pred = cross_val_predict(SVM,X,y,cv=5)
sns.heatmap(confusion_matrix(y,y_pred),annot=True,fmt='3.0f',cmap="summer")
plt.title('Confusion_matrix(SVM)', y=1.05, size=15)

auc

In [ ]:
import matplotlib.patches as patches
from scipy import interp
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve,auc
from sklearn.model_selection import StratifiedKFold


cv = StratifiedKFold(n_splits=5,shuffle=False)

# plot arrows
fig1 = plt.figure(figsize=[7,7])
ax1 = fig1.add_subplot(111,aspect = 'equal')

tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in cv.split(X_,y_):
    prediction = RF.fit(X_.iloc[train],y_.iloc[train]).predict_proba(X_.iloc[test])
    fpr, tpr, t = roc_curve(y_[test], prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC(RF)')
plt.legend(loc="lower right")
plt.show()